In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic")
sc = SparkContext(conf=conf)
sc

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 09:21:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=spark_sql_basic>

In [2]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])

attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

마블 영화 중 관객수가 500만 이상인 영화 가져오기

In [4]:
# CASE 1 : Join을 먼저, Filter 나중에
movie_att_rdd = movies_rdd.join(attendances_rdd)
movie_att_rdd.filter(lambda x : x[1][0][1] == '마블' and x[1][1][0] >= 5000000).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [5]:
# CASE 2 : Filter 먼저, Join을 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == "마블")
filtered_attendances = attendances_rdd.filter(lambda x : x[1][0] >= 5000000)

filtered_movies.join(filtered_attendances).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

동일한 결과가 나오긴 하지만 filter를 먼저 수행해서 가져올 데이터를 걸러 낸 다음, join을 하는 CASE 2가 훨씬 효율적이다.
<pre>
매번 이런 고민을 하기엔 시간이 너무 아깝고 개발자 마다 성능 차이도 심하다.
</pre>

In [6]:
sc.stop()

# 1. SparkSession 생성하기
- `SparkContext`에 해당하며, 새로운 스파크 어플리케이션을 만들어준다.

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()
spark

In [8]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [9]:
movie_schema = ["id", "name", "company", "year", "month", "day"]

# 2. 데이터 프레임 생성

In [10]:
movie_sdf = spark.createDataFrame(data=movies, schema=movie_schema)
movie_sdf

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [11]:
# 스키마 정보 확인
movie_sdf.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [12]:
movie_sdf.show()  # show() = action

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# 3. Spark SQL 사용하기
- DataFrame을 임시 뷰로 해줘야 Spark SQL을 사용할 수 있음
- `createOrReplaceTempView` 함수를 이용해서 DataFrame에 SQL을 사용할 수 있는 View를 만들어 준다.

In [13]:
movie_sdf.createOrReplaceTempView("movies")

In [15]:
query = """

    SELECT name
    FROM movies

"""

# 쿼리 실행
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [16]:
# 영화 이름, 개봉 연도 가져오기
query = """

    SELECT name, year
    FROM movies

"""

spark.sql(query).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|2012|
|  슈퍼맨|2013|
|  배트맨|2008|
|겨울왕국|2014|
|아이언맨|2008|
+--------+----+



In [17]:
# 2010년도 이후에 개봉한 영화의 모든 정보
query = """

    SELECT *
    FROM movies
    WHERE year >= 2010

"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [19]:
# 2010년도 이후에 개봉한 마블 영화의 모든 정보
query = """

    SELECT *
    FROM movies
    WHERE year >= 2010 and company = "마블"

"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



In [20]:
# ~맨으로 끝나는 영화의 모든 정보
query = """

    SELECT *
    FROM movies
    WHERE name LIKE "%맨"

"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [21]:
# id가 3번인 영화보다 늦게 개봉한 마블영화의 모든 정보(연도만 고려)
query = """

    SELECT *
    FROM movies
    WHERE company = '마블' and year > (SELECT year
                                       FROM movies
                                       WHERE id == 3)
"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



## JOIN 구현

In [22]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [23]:
# 자료형 타입 불러오기
from pyspark.sql.types import StringType, FloatType, IntegerType

# 구조를 만들기 위한 타입 불러오기(필수). 컬럼 순서 등
from pyspark.sql.types import StructField, StructType

In [24]:
att_schema = StructType([
    StructField("id", IntegerType(), True),  # True : Not Null 의미
    StructField("attendance", FloatType(), True),
    StructField("country", StringType(), True)
])

In [25]:
att_df = spark.createDataFrame(data=attendances, schema=att_schema)
att_df

DataFrame[id: int, attendance: float, country: string]

In [26]:
att_df.createOrReplaceTempView("att")

In [27]:
# movies , att id를 기반으로 JOIN
query = """

    SELECT *
    FROM movies, att
    WHERE movies.id = att.id

"""

spark.sql(query).show()

+---+--------+-------+----+-----+---+---+-----------+-------+
| id|    name|company|year|month|day| id| attendance|country|
+---+--------+-------+----+-----+---+---+-----------+-------+
|  1|어벤져스|   마블|2012|    4| 26|  1|1.3934592E7|     KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2|  2182227.0|     KR|
|  3|  배트맨|     DC|2008|    8|  6|  3|  4226242.0|     KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|  4|1.0303058E7|     KR|
|  5|아이언맨|   마블|2008|    4| 30|  5|  4300365.0|     KR|
+---+--------+-------+----+-----+---+---+-----------+-------+



In [28]:
spark.stop()